[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/etf-fund-holdings-python.ipynb)

# ETF and Fund Holdings from SEC EDGAR with Python -- Free, No API Key

Use **edgartools** to analyze ETF and mutual fund portfolio holdings from SEC NPORT filings -- completely free, no API key or paid subscription required. Every fund registered with the SEC must file NPORT-P reports disclosing their complete portfolio.

**What you'll learn:**
- Get any ETF or fund's complete portfolio holdings
- See top holdings, portfolio weight, and total assets
- Analyze asset allocation across equities, bonds, and derivatives
- Compare holdings across multiple funds
- Export holdings to pandas DataFrames

## Install edgartools

In [ ]:
!pip install -U edgartools

## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [ ]:
import pandas as pd
from edgar import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## See SPY's Complete Portfolio in 3 Lines

Every ETF files NPORT-P reports with the SEC disclosing their complete holdings. Get SPY's portfolio instantly:

In [ ]:
fund = Company("SPY").get_filings(form="NPORT-P")[0].obj()

print(f"Fund:       {fund.name}")
print(f"Period:     {fund.reporting_period}")
print(f"Holdings:   {len(fund.investments):,}")
print(f"Net Assets: ${fund.fund_info.net_assets:,.0f}")

## Top 10 Holdings by Value

Use `investment_data()` to get all holdings as a pandas DataFrame, then sort by portfolio weight:

In [ ]:
df = fund.investment_data()
df["value_usd"] = pd.to_numeric(df["value_usd"])
df["pct_value"] = pd.to_numeric(df["pct_value"])

top10 = df.nlargest(10, "value_usd")[["name", "ticker", "value_usd", "pct_value"]].copy()
top10["Value ($B)"] = (top10["value_usd"] / 1e9).round(1)
top10["Weight %"] = top10["pct_value"].round(2)
top10[["name", "ticker", "Value ($B)", "Weight %"]].reset_index(drop=True)

## Analyze Any ETF: QQQ (Nasdaq 100)

The same approach works for any ETF. Here's the Invesco QQQ Trust:

In [ ]:
qqq = Company("QQQ").get_filings(form="NPORT-P")[0].obj()

print(f"Fund:     {qqq.name}")
print(f"Holdings: {len(qqq.investments)}")
print(f"Net Assets: ${qqq.fund_info.net_assets:,.0f}")

qqq_df = qqq.investment_data()
qqq_df["value_usd"] = pd.to_numeric(qqq_df["value_usd"])
qqq_df["pct_value"] = pd.to_numeric(qqq_df["pct_value"])

print(f"\nTop 5 holdings:")
for _, row in qqq_df.nlargest(5, "value_usd").iterrows():
    print(f"  {row['name']:30s} {str(row['ticker']):6s} {row['pct_value']:.1f}%")

## Compare Holdings Across ETFs

Find which stocks appear in multiple ETFs and compare their weights:

In [ ]:
# Get top 10 from each fund
spy_top = df.nlargest(10, "value_usd")[["ticker", "pct_value"]].set_index("ticker")
spy_top.columns = ["SPY %"]

qqq_top = qqq_df.nlargest(10, "value_usd")[["ticker", "pct_value"]].set_index("ticker")
qqq_top.columns = ["QQQ %"]

# Merge to compare
comparison = spy_top.join(qqq_top, how="outer").round(2)
comparison = comparison.sort_values("SPY %", ascending=False)
comparison

## Bond Fund Holdings

NPORT filings work for bond funds too. Here's Vanguard Total Bond Market:

In [ ]:
bnd = Company("BND").get_filings(form="NPORT-P")[0].obj()

print(f"Fund:     {bnd.name}")
print(f"Holdings: {len(bnd.investments):,}")
print(f"Net Assets: ${bnd.fund_info.net_assets:,.0f}")

bnd_df = bnd.investment_data()
bnd_df["value_usd"] = pd.to_numeric(bnd_df["value_usd"])

# Asset type breakdown
asset_types = bnd_df.groupby("asset_category")["value_usd"].sum().sort_values(ascending=False)

print(f"\nAsset Allocation:")
categories = {"DBT": "Bonds", "ABS-MBS": "Mortgage-Backed", "STIV": "Short-Term", "ABS-CBDO": "CDOs"}
for cat, val in asset_types.items():
    label = categories.get(cat, cat)
    print(f"  {label:25s} ${val/1e9:>8.1f}B")

## Search for Any Fund's NPORT Filing

Use `get_filings()` to find NPORT filings across all funds:

In [ ]:
# Recent NPORT filings across all funds
nport_filings = get_filings(form="NPORT-P")
print(f"Recent NPORT-P filings: {len(nport_filings):,}")
nport_filings.head(5)

## Why EdgarTools?

EdgarTools is free and open-source. Compare accessing ETF holdings:

**With edgartools (free, no API key):**
```python
fund = Company("SPY").get_filings(form="NPORT-P")[0].obj()
holdings = fund.investment_data()   # Complete portfolio as DataFrame
```

**Typical paid API approach ($50+/month, API key required):**
```python
from sec_api import QueryApi
api = QueryApi(api_key="YOUR_PAID_API_KEY")
query = {"query": {"query_string": {"query": 'formType:"NPORT-P" AND ticker:"SPY"'}}}
results = api.get_filings(query)  # Raw JSON, still need to parse XML manually
```

With edgartools, fund holdings are parsed into clean DataFrames automatically -- no XML parsing, no API key, no monthly fee.

## Quick Reference

```python
import pandas as pd
from edgar import *
set_identity("your.name@example.com")

# ── Get fund holdings ──
fund = Company("SPY").get_filings(form="NPORT-P")[0].obj()

# ── Fund metadata ──
fund.name                       # Fund name
fund.reporting_period            # Report date
fund.fund_info.net_assets        # Net assets
fund.fund_info.total_assets      # Total assets
len(fund.investments)            # Number of holdings

# ── Holdings as DataFrame ──
df = fund.investment_data()
df["value_usd"] = pd.to_numeric(df["value_usd"])
df["pct_value"] = pd.to_numeric(df["pct_value"])
df.nlargest(10, "value_usd")     # Top 10 holdings

# ── Specialized data ──
fund.securities_data()           # Non-derivative securities
fund.derivatives_data()          # Derivatives only

# ── Search NPORT filings ──
get_filings(form="NPORT-P")      # Recent NPORT filings
```

## What's Next

You've learned how to analyze ETF and fund holdings with Python. Here are related tutorials:

- [Analyze 13F Institutional Holdings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/13f-institutional-holdings-python.ipynb)
- [Extract Revenue and Earnings from SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/extract-revenue-earnings-python.ipynb)
- [Compare Company Financials](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/compare-company-financials-python.ipynb)
- [Download SEC Filings in Bulk](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/download-sec-filings-bulk-python.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who works with SEC data? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*